## Imports

In [ ]:
import random
import os
import numpy as np
import matplotlib.pyplot as plt

from pycs.astro.wl.mass_mapping import *
from pycs.sparsity.sparse2d.starlet import *
from pycs.misc.cosmostat_init import *
from pycs.astro.wl.hos_peaks_l1 import *

from sp_peaks import slics
from sp_peaks import mapping
from sp_peaks import summary_statistics
from sp_peaks import plotting
from multiprocessing import Pool

## Bookkeeping

Make master file

In [ ]:
# Define the root directory
root_directory = "/n17data/tersenov/Cov_DES_SLICS"

# Open the master text file in write mode
master_file_path = ".././input/master_file_cov.txt"
with open(master_file_path, "w") as master_file:
    for subdir in os.listdir(root_directory):
        # Check if the subdirectory name matches the pattern "LOS..."
        if subdir.startswith("LOS") and subdir[3:].isdigit():
            subdir_path = os.path.join(root_directory, subdir)
            
            # Iterate over the files in the subdirectory
            for file_name in os.listdir(subdir_path):
                # Check if the file name matches the desired pattern
                if file_name.startswith("DES_MocksCat_SLICS_4_Bin") and file_name.endswith(".dat"):
                    file_path = os.path.join(subdir_path, file_name)
                    master_file.write(file_path + "\n")

In [ ]:
# Define the path to the "master_file_cov.txt"
master_file_path = ".././input/master_file_cov.txt"

Parse the filenames

In [ ]:
# Read the file paths from the "master_file_cov.txt"
with open(master_file_path, "r") as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

# Parse these file paths
parsed_cov_data = slics.parse_cov_SLICS_filenames(file_paths)

Reconstruct 124 realisations of the survey by picking each tile from a random LOS, ensuring that each file is only included once.

In [ ]:
los_numbers = np.unique(parsed_cov_data['LOS']) # List of all LOS numbers
num_realizations = 124 # Number of realizations
num_tiles_per_realization = 19 # Number of tiles to select for each realization

num_bins = 4
bin_number = 2

# Reconstruct 124 realisations of the survey by picking each tile from a random LOS, ensuring that each file is only included once.
collections_of_files = slics.survey_realizations_reconstruction(num_realizations, num_tiles_per_realization, bin_number, parsed_cov_data['LOS'], file_paths)

## Peak counts datavector calculation

In [ ]:
# Constants and Parameters
N_GAL = 7 
SIZE_X_DEG = 10.
SIZE_Y_DEG = 10.
PIX_ARCMIN = 1.
SHAPE_NOISE = 0.44
NSCALES = 5

# Histogram parameters
MIN_SNR = -2
MAX_SNR = 6
NBINS=31

NUM_REALIZATIONS = 124  # Number of realizations

In [ ]:
# Initialize an empty list to store data vectors for each realization
data_vectors = []

# Loop over realizations
for realization_files in collections_of_files[:3]: # Use only the first ... realizations for now
    # Initialize an empty list to store peak counts vectors for each tile in this realization
    peak_counts_realization = []

    # Loop over files (tiles) in this realization
    for tile_file in realization_files:
        # Load the catalog data for this tile
        catalog_data = slics.read_catalogue_pd(tile_file)

        # Extract data from the catalog
        ra = catalog_data['RA']
        dec = catalog_data['Dec']
        g1_sim = catalog_data['gamma1_sim']
        g2_sim = catalog_data['gamma2_sim']

        # Calculate peak counts for this tile
        x, y = radec2xy(np.mean(ra), np.mean(dec), ra, dec)
        Nx, Ny = int(SIZE_X_DEG / PIX_ARCMIN * 60), int(SIZE_Y_DEG / PIX_ARCMIN * 60)
        galmap = bin2d(x, y, npix=(Nx,Ny))
        mask = (galmap > 0).astype(int)

        sigma_noise = np.zeros_like(galmap)
        sigma_noise[mask != 0] = SHAPE_NOISE / np.sqrt(2 * galmap[mask != 0])
        sigma_noise[mask == 0] = np.max(sigma_noise[mask != 0]) # set the noise to the maximum value in the map where there are galaxies        noise_map_CFIS_z05 = sigma_noise * np.random.randn(sigma_noise.shape[0], sigma_noise.shape[1]) # generate noise map
        noise_map_CFIS_z05 = sigma_noise * np.random.randn(sigma_noise.shape[0], sigma_noise.shape[1]) # generate noise map

        e1map, e2map = bin2d(x, y, npix=(Nx, Ny), v=(g1_sim, g2_sim)) 

        # Shear data class initialization
        d = shear_data()
        d.g1 = e1map
        d.g2 = -e2map
        (nx,ny) = e1map.shape
        d.mask = mask
        # Shear noise covariance matrix
        Ncov = np.zeros((nx,ny))
        Ncov[mask > 0] = 2. * sigma_noise[mask > 0]**2
        Ncov[mask == 0] = 1e9 # set the noise to the maximum value in the map where there are galaxies

        d.Ncov = Ncov
        d.nx = nx
        d.ny = ny  

        # Mass mapping class initialization
        M = massmap2d(name='mass')
        M.init_massmap(d.nx,d.ny)
        M.DEF_niter = 50
        Inpaint = True
        M.niter_debias = 30
        M.Verbose = False 

        # KS
        ks =  M.gamma_to_cf_kappa(e1map,-e2map) 
        ks = ks.real
        ks_noisy = ks + noise_map_CFIS_z05

        WT = starlet2d(gen2=False,l2norm=False, verb=False)
        WT.init_starlet(nx, ny, nscale=NSCALES)

        H = HOS_starlet_l1norm_peaks(WT)
        H.set_bins(Min=MIN_SNR, Max=MAX_SNR, nbins=NBINS)
        H.get_mono_scale_peaks(ks_noisy, sigma_noise, mask=mask)

        peak_counts_single = H.Mono_Peaks_Count

        # Append peak counts for this tile to the list
        peak_counts_realization.append(peak_counts_single)

    # Compute the average peak counts vector for this realization
    average_peak_counts = np.mean(peak_counts_realization, axis=0)

    # Append the average peak counts vector for this realization to the list of data vectors
    data_vectors.append(average_peak_counts)

# Convert the list of data vectors into a NumPy array
data_vectors = np.array(data_vectors)

## Peak counts covariance calculation

In [ ]:
# Compute the average peak counts vector across all realizations
mean_PC_over_realizations = np.mean(data_vectors, axis=0)

# Compute the deviations of peak counts in each realization from the average vector
deviations = data_vectors - mean_PC_over_realizations

# Compute the covariance matrix
# The covariance_matrix will be a square matrix of shape (num_bins, num_bins).
num_realizations = data_vectors.shape[0]
covariance_matrix = np.dot(deviations.T, deviations) / (num_realizations - 1)

# # Normalize the covariance matrix to the unity of the diagonal
# diagonal_sqrt = np.sqrt(np.diag(covariance_matrix))
# covariance_matrix_normalized = covariance_matrix / np.outer(diagonal_sqrt, diagonal_sqrt)

# Normalize the covariance matrix
max_cov_value = np.max(covariance_matrix)
covariance_matrix_normalized = covariance_matrix / max_cov_value

In [ ]:
plotting.plot_map(covariance_matrix_normalized, title='Covariance Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

In [ ]:
correlation_matrix_SS_PC = np.corrcoef(covariance_matrix)
plotting.plot_map(correlation_matrix_SS_PC, title='Correlation Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

## Multiscale peak counts datavector

In [ ]:
# Constants and Parameters
N_GAL = 7 
SIZE_X_DEG = 10.
SIZE_Y_DEG = 10.
PIX_ARCMIN = 1.
SHAPE_NOISE = 0.44
NSCALES = 5

# Histogram parameters
MIN_SNR = -2
MAX_SNR = 6
NBINS=31

NUM_REALIZATIONS = 124  # Number of realizations

In [ ]:
# Initialize an empty list to store data the Multi-Scale Peak Counts vectors for each realization
MS_PC_data_vectors = []

# Loop over realizations
for realization_files in collections_of_files[:3]:  # Use only the first ... realizations for now
    # Initialize an empty list to store peak counts vectors for each scale in this realization
    peak_counts_realization = []

    # Loop over files (tiles) in this realization
    for tile_file in realization_files:
        # Load the catalog data for this tile
        catalog_data = slics.read_catalogue_pd(tile_file)

        # Extract data from the catalog
        ra = catalog_data['RA']
        dec = catalog_data['Dec']
        g1_sim = catalog_data['gamma1_sim']
        g2_sim = catalog_data['gamma2_sim']

        # Calculate peak counts for this tile
        x, y = radec2xy(np.mean(ra), np.mean(dec), ra, dec)
        Nx, Ny = int(SIZE_X_DEG / PIX_ARCMIN * 60), int(SIZE_Y_DEG / PIX_ARCMIN * 60)
        galmap = bin2d(x, y, npix=(Nx,Ny))
        mask = (galmap > 0).astype(int)
        sigma_noise = np.zeros_like(galmap)
        sigma_noise[mask != 0] = SHAPE_NOISE / np.sqrt(2 * galmap[mask != 0])
        sigma_noise[mask == 0] = np.max(sigma_noise[mask != 0]) # set the noise to the maximum value in the map where there are galaxies        noise_map_CFIS_z05 = sigma_noise * np.random.randn(sigma_noise.shape[0], sigma_noise.shape[1]) # generate noise map
        noise_map_CFIS_z05 = sigma_noise * np.random.randn(sigma_noise.shape[0], sigma_noise.shape[1]) # generate noise map
        e1map, e2map = bin2d(x, y, npix=(Nx, Ny), v=(g1_sim, g2_sim)) 

        # Shear data class initialization
        d = shear_data()
        d.g1 = e1map
        d.g2 = -e2map
        (nx,ny) = e1map.shape
        d.mask = mask
        # Shear noise covariance matrix
        Ncov = np.zeros((nx,ny))
        Ncov[mask > 0] = 2. * sigma_noise[mask > 0]**2
        Ncov[mask == 0] = 1e9 # set the noise to the maximum value in the map where there are galaxies

        d.Ncov = Ncov
        d.nx = nx
        d.ny = ny  

        # Mass mapping class initialization
        M = massmap2d(name='mass')
        M.init_massmap(d.nx,d.ny)
        M.DEF_niter = 50
        Inpaint = True
        M.niter_debias = 30
        M.Verbose = False 

        # KS
        ks =  M.gamma_to_cf_kappa(e1map,-e2map) 
        ks = ks.real
        ks_noisy = ks + noise_map_CFIS_z05

        WT = starlet2d(gen2=False,l2norm=False, verb=False)
        WT.init_starlet(nx, ny, nscale=NSCALES)

        H = HOS_starlet_l1norm_peaks(WT)
        H.set_bins(Min=MIN_SNR, Max=MAX_SNR, nbins=NBINS)
        H.set_data(ks_noisy, SigmaMap=sigma_noise, Mask=mask)
        H.get_wtpeaks(Mask=mask)
        peak_counts_multi = H.Peaks_Count

        # Append peak counts for this tile to the list for this realization
        peak_counts_realization.append(peak_counts_multi)

    # Compute the average peak counts vector for this realization
    average_peak_counts = np.mean(peak_counts_realization, axis=0)

    # Append the average peak counts vector for this realization to the list of data vectors
    MS_PC_data_vectors.append(average_peak_counts)

# Convert the list of data vectors into a NumPy array
MS_PC_data_vectors = np.array(MS_PC_data_vectors)

In [ ]:
# load the data vectors from a file
MS_PC_data_vectors = np.load('.././output/MS_PC_data_vectors.npy')

In [ ]:
data_vectors_reshaped = MS_PC_data_vectors.reshape(10, -1)  

# Compute the average peak counts vector across all realizations for multiscale peak counts
mean_PC_over_realizations_multi = np.mean(data_vectors_reshaped, axis=0)

# Compute the deviations of peak counts in each realization from the average vector
deviations_multi = data_vectors_reshaped - mean_PC_over_realizations_multi

# Compute the covariance matrix for multiscale peak counts
num_realizations_multi, num_bins_multi = data_vectors_reshaped.shape
covariance_matrix_multi = np.dot(deviations_multi.T, deviations_multi) / (num_realizations_multi - 1)

# Normalize the covariance matrix for multiscale peak counts
diagonal_multi = np.sqrt(np.diag(covariance_matrix_multi))
# Add a small epsilon to the diagonal to avoid division by zero
epsilon = 1e-10
covariance_matrix_normalized_multi = covariance_matrix_multi / (np.outer(diagonal_multi, diagonal_multi)+epsilon)

In [ ]:
plotting.plot_map(covariance_matrix_multi, title='Covariance Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

In [ ]:
plotting.plot_map(covariance_matrix_normalized_multi, title='Covariance Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

In [ ]:
correlation_matrix_MS_PC = np.corrcoef(covariance_matrix_multi.T)
plotting.plot_map(correlation_matrix_MS_PC, title='Correlation Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

## l1-norm datavector

In [ ]:
# Constants and Parameters
N_GAL = 7 
SIZE_X_DEG = 10.
SIZE_Y_DEG = 10.
PIX_ARCMIN = 1.
SHAPE_NOISE = 0.44
NSCALES = 5

# Histogram parameters
MIN_SNR = -2
MAX_SNR = 6
NBINS_L1 = 40

NUM_REALIZATIONS = 124  # Number of realizations

In [ ]:
# Initialize an empty list to store data vectors for the L1-norm histogram for each realization
L1_norm_data_vectors = []

# Loop over realizations
for realization_files in collections_of_files[:10]:  # Use only the first ... realizations for now
    # Initialize an empty list to store L1-norm histogram vectors for each scale in this realization
    l1_norm_histogram_realization = []

    # Loop over files (tiles) in this realization
    for tile_file in realization_files:
        # Load the catalog data for this tile
        catalog_data = slics.read_catalogue_pd(tile_file)

        # Extract data from the catalog
        ra = catalog_data['RA']
        dec = catalog_data['Dec']
        g1_sim = catalog_data['gamma1_sim']
        g2_sim = catalog_data['gamma2_sim']

        # Calculate peak counts for this tile
        x, y = radec2xy(np.mean(ra), np.mean(dec), ra, dec)
        Nx, Ny = int(SIZE_X_DEG / PIX_ARCMIN * 60), int(SIZE_Y_DEG / PIX_ARCMIN * 60)
        galmap = bin2d(x, y, npix=(Nx,Ny))
        mask = (galmap > 0).astype(int)
        sigma_noise = np.where(mask != 0, SHAPE_NOISE / np.sqrt(2 * galmap), np.max(sigma_noise[mask != 0]))
        noise_map_CFIS_z05 = sigma_noise * np.random.randn(sigma_noise.shape[0], sigma_noise.shape[1]) # generate noise map
        e1map, e2map = bin2d(x, y, npix=(Nx, Ny), v=(g1_sim, g2_sim)) 

        # Shear data class initialization
        d = shear_data()
        d.g1 = e1map
        d.g2 = -e2map
        (nx,ny) = e1map.shape
        d.mask = mask
        # Shear noise covariance matrix
        Ncov = np.zeros((nx,ny))
        Ncov[mask > 0] = 2. * sigma_noise[mask > 0]**2
        Ncov[mask == 0] = 1e9 # set the noise to the maximum value in the map where there are galaxies

        d.Ncov = Ncov
        d.nx = nx
        d.ny = ny  

        # Mass mapping class initialization
        M = massmap2d(name='mass')
        M.init_massmap(d.nx,d.ny)
        M.DEF_niter = 50
        Inpaint = True
        M.niter_debias = 30
        M.Verbose = False 

        # KS
        ks =  M.gamma_to_cf_kappa(e1map,-e2map) 
        ks = ks.real
        ks_noisy = ks + noise_map_CFIS_z05

        WT = starlet2d(gen2=False,l2norm=False, verb=False)
        WT.init_starlet(nx, ny, nscale=NSCALES)

        H = HOS_starlet_l1norm_peaks(WT)
        H.set_data(ks_noisy, SigmaMap=sigma_noise, Mask=mask)
        H.get_wtl1(NBINS_L1*2, Mask=mask)
        l1norm_histogram = H.l1norm

        # Append the L1-norm histogram for this tile to the list for this realization
        l1_norm_histogram_realization.append(l1norm_histogram)

    # Compute the average L1-norm histogram vector for this realization
    average_l1_norm_histogram = np.mean(l1_norm_histogram_realization, axis=0)

    # Append the average L1-norm histogram vector for this realization to the list of data vectors
    L1_norm_data_vectors.append(average_l1_norm_histogram)

# Convert the list of data vectors into a NumPy array
L1_norm_data_vectors = np.array(L1_norm_data_vectors)


In [ ]:
# load the data vectors from a file
L1_norm_data_vectors = np.load('.././output/L1_norm_data_vectors.npy')

In [ ]:
L1_norm_data_vectors_reshaped = L1_norm_data_vectors.reshape(10, -1)

# Compute the average L1-norm histogram vector across all realizations for the L1-norm
mean_L1_norm_over_realizations = np.mean(L1_norm_data_vectors_reshaped, axis=0)

# Compute the deviations of L1-norm histograms in each realization from the average vector
deviations_L1_norm = L1_norm_data_vectors_reshaped - mean_L1_norm_over_realizations

# Compute the covariance matrix for the L1-norm histograms
num_realizations_L1_norm, num_bins_L1_norm = L1_norm_data_vectors_reshaped.shape
covariance_matrix_L1_norm = np.dot(deviations_L1_norm.T, deviations_L1_norm) / (num_realizations_L1_norm - 1)

# Calculate the diagonal of the covariance matrix
diagonal_L1_norm = np.sqrt(np.diag(covariance_matrix_L1_norm))

# Calculate the correlation coefficients
correlation_matrix_L1_norm = covariance_matrix_L1_norm / np.outer(diagonal_L1_norm, diagonal_L1_norm)

In [ ]:
plotting.plot_map(covariance_matrix_L1_norm, title='Covariance Matrix of L1-norm Histograms', cmap='viridis', vmin=None, vmax=None)

In [ ]:
plotting.plot_map(correlation_matrix_L1_norm, title='Correlation Matrix of L1-norm Histograms', cmap='viridis', vmin=None, vmax=None)

In [ ]:
correlation_matrix_L1_norm = np.corrcoef(covariance_matrix_L1_norm.T)
plotting.plot_map(correlation_matrix_L1_norm, title='Correlation Matrix of L1-norm Histograms', cmap='viridis', vmin=None, vmax=None)

In [ ]:
SS_PC_data_vectors = np.load('/home/tersenov/shear-pipe-peaks/output/data_vector_SS_PC_cs.npy')
MS_PC_data_vectors = np.load('/home/tersenov/shear-pipe-peaks/output/MS_PC_data_vectors.npy')
l1_norm_data_vectors = np.load('/home/tersenov/shear-pipe-peaks/output/L1_norm_data_vectors.npy')

In [ ]:
mean_PC_over_realizations = np.mean(SS_PC_data_vectors, axis=0)
deviations = SS_PC_data_vectors - mean_PC_over_realizations

num_realizations, num_bins = SS_PC_data_vectors.shape
covariance_matrix = np.dot(deviations.T, deviations) / (num_realizations - 1)

# max_cov_value = np.max(covariance_matrix)
# covariance_matrix_normalized = covariance_matrix / max_cov_value



diagonal_SS_PC = np.sqrt(np.diag(covariance_matrix))
diagonal_SS_PC[diagonal_SS_PC == 0] = 1e-10
correlation_matrix_SS_PC = covariance_matrix / np.outer(diagonal_SS_PC, diagonal_SS_PC)

In [ ]:
correlation_matrix_PC = np.corrcoef(SS_PC_data_vectors, rowvar=False)

In [ ]:
plotting.plot_map(correlation_matrix_PC, title='Covariance Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

In [ ]:
plotting.plot_map(covariance_matrix_normalized, title='Covariance Matrix of Peak Counts', cmap='viridis', vmin=None, vmax=None)

In [ ]:
MS_PC_data_vectors_reshaped = MS_PC_data_vectors.reshape(10, -1)
mean_MS_PC_over_realizations = np.mean(MS_PC_data_vectors_reshaped, axis=0)
deviations_MS_PC = MS_PC_data_vectors_reshaped - mean_MS_PC_over_realizations
num_realizations_MS_PC, num_bins_MS_PC = MS_PC_data_vectors_reshaped.shape
covariance_matrix_MS_PC = np.dot(deviations_MS_PC.T, deviations_MS_PC) / (num_realizations_MS_PC - 1)
diagonal_MS_PC = np.sqrt(np.diag(covariance_matrix_MS_PC))
diagonal_MS_PC[diagonal_MS_PC == 0] = 1e-10
correlation_matrix_MS_PC = covariance_matrix_MS_PC / np.outer(diagonal_MS_PC, diagonal_MS_PC)
plotting.plot_map(correlation_matrix_MS_PC, title='MS-PC', cmap='viridis', vmin=None, vmax=None)

In [ ]:
correlation_matrix_MS_PC = np.corrcoef(MS_PC_data_vectors_reshaped, rowvar=False)
plotting.plot_map(correlation_matrix_MS_PC, title='MS-PC', cmap='viridis', vmin=None, vmax=None)

In [ ]:
l1_norm_data_vectors_reshaped = l1_norm_data_vectors.reshape(10, -1)
mean_l1_norm_over_realizations = np.mean(l1_norm_data_vectors_reshaped, axis=0)
deviations_l1_norm = l1_norm_data_vectors_reshaped - mean_l1_norm_over_realizations
num_realizations_l1_norm, num_bins_l1_norm = l1_norm_data_vectors_reshaped.shape
covariance_matrix_l1_norm = np.dot(deviations_l1_norm.T, deviations_l1_norm) / (num_realizations_l1_norm - 1)
diagonal_l1_norm = np.sqrt(np.diag(covariance_matrix_l1_norm))
diagonal_l1_norm[diagonal_l1_norm == 0] = 1e-10
correlation_matrix_l1_norm = covariance_matrix_l1_norm / np.outer(diagonal_l1_norm, diagonal_l1_norm)

In [ ]:
plotting.plot_map(correlation_matrix_l1_norm, title='l1-norm', cmap='viridis', vmin=None, vmax=None)

In [ ]:
correlation_matrix_l1_norm = np.corrcoef(l1_norm_data_vectors_reshaped, rowvar=False)

In [ ]:
plotting.plot_map(correlation_matrix_l1_norm, title='l1-norm', cmap='viridis', vmin=None, vmax=None)